In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from fireworks.nbodylib.integrators import integrator_leapfrog, integrator_leapfrog_galaxy, integrator_rungekutta, integrator_tsunami
from fireworks.nbodylib.dynamics import acceleration_pyfalcon, acceleration_jerk_direct
from fireworks.nbodylib.timesteps import adaptive_timestep_r
from fireworks.particles import Particles
from typing import Optional, Tuple, Callable, Union
import time

#from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

# N-Body

### N-body problem

The N-body problem is the problem concerning the prediction of the motion of a group of objects interacting gravitationally with eachother. This conudrum [problem] is of paramount importance in astrophysics because is applied to a huge plethora of astrophysical situations and scales (Solar system, satellites, binary evolution, stars in a cluster or in a galaxy...). \
It dates back to the XVII century when Isaac Newton came up with the intuition that two bodies with mass $m_1$ and $m_2$ at a distance $r_{12}$ from eachother are subjected to a mutual force
\begin{equation}
    \vec{F}_{1,2} = m_1 \vec{a}_1 = - G \frac{m_1 m_2}{r_{12}^2} \frac{ \vec{r}_{12} }{ r_{12} } 
\end{equation}
Then, extending this to a system of N-body, the acceleration of the body $i$ due to all the other bodies is expressed by 
\begin{equation}
    \frac{ d^2 \vec{x}_i } {dt^2} = - G \sum^N_{j=1, j\neq i} m_j \frac{ \vec{x}_i - \vec{x}_j }{ | \vec{x}_i - \vec{x}_j |^3  }
\end{equation}

It exists an analitic solution for $N=2$ (Bernoulli was the first to derive it in 1710). The complete solution for a generic number of bodies had not been found yet (in 1991 Qiudong Wang found a convergent power series solution for a generic number of bodies, but is too difficult to implement and has a slow convergence). Therefore, we use numerical methods in order to integrate this system of differential equation. 

Another important thing to point out is the numeical complexity of this problem, which is nothing but the number of calculations (numerical iterations) needed to perform the task. In the case examined the time complecity is $O(N^2)$: it grows fastly as N increases, and this represent a big problem do tackle when constructing a model for the system. 

### N-Body units


N-body units are a very powerful and useful method when dealing with the N-body problem. They represent a convenient set of units for N-body simulations, based on the assumption that $G = 1$. \
The conversion to physical values can be done a-posteriori, inserving eventually some typical values of the astrophysical system of interest. \
It must be kept in mind that this treatment fails when adding to the simulation data about stars' proper mass and radius, or stellar evolution, or SN explosion, or when setting additional fields in order to reproduce a certan galactic physical configuration; nevertheless, in our code we treat particles as abstract point mass stars, and therefore the scale invariance of our N-body simulation can be exploited.

In order to convert N-body units into physical units we have to define a scale lenght $L_{scale}$ and a scale mass $M_{scale}$. Then I have:
\begin{equation}
    T_{scale} = \sqrt{  \frac{L_{scale}^3}{G M_{scale}}   }   ,
\end{equation}

\begin{equation}
    V_{scale} = \frac{L_{scale}}{T_{scale}} = \sqrt{  \frac{G M_{scale}}{L_{scale}}   } 
\end{equation}
And from here we can ecover the physical units

\begin{equation}
    L_{phys}= L_{Nbody} L_{scale}
\end{equation}

\begin{equation}
    M_{phys}= M_{Nbody} M_{scale}
\end{equation}

\begin{equation}
    T_{phys}= T_{Nbody} T_{scale}
\end{equation}

\begin{equation}
    V_{phys}= V_{Nbody} V_{scale}
\end{equation}


Anyway, we can easily recover physical units with the Fireworks package.

# Collisional vs Collisionless Simulations

The importance of close particle-particle interactions allow us to distinguish between Collisional and Collisionless systems. In particular we start analyzing if and how much gravitational encounters between stars are able to change the kinematic status of themselves. \
To tackle this problem we consider a test star with velocity $v$ which crosses the potential well of another star a rest. Both the stars have the same mass $m$, and the mutual distance is $b$. After some calculation, the velocity change of the incoming star is $\delta v = \frac{2 G m}{b v}$. If we consider all the stars in the galaxy we can easily see that the average change $\delta v$ is zero, but the standard deviation is not. In particular one can find out that 
\begin{equation}
    \frac{ \Delta v^2 }{v^2} \approx 8 \frac{1}{N} log\frac{N}{2}
\end{equation}
This implies that to produce a substantial change in the velocity the star needs to cross the galaxy a number of times $n_{relax}$ equal to 
\begin{equation}
    n_{relax} \approx  \frac{N}{ 8 log \frac{N}{2} } 
\end{equation}
From here, defining $t_{cross} \approx \frac{R}{v}$ as the time required approximately from the star to cross the whole cluster, we can define the relaxation time scale as 
\begin{equation}
    t_{relax} = n_{relax} t_{cross}
\end{equation}
Substituting in this equation the previous expressions I finally get 
\begin{equation}
    t_{relax} \approx 0.1 \frac{N}{log N } t_{cross}
\end{equation}
Overall, we can use this time to distinghish between collisional and collisionless systems. In particular:

1) $t_{simulation} \gtrsim t_{relax}$: COLLISIONAL SYSTEMS \
In this case particle-particle interactions cannot be neglected if we want to describe particles' motion: we have to go through a direct force extimate.
2) $t_{simulation} \ll t_{relax}$: COLLISIONLESS SYSTEMS \
In this case particle-particle interactions do not have an important role, and we can track particles' trajectories considering a smooth matter distribution: we can use less computationally expensive tecniques to estimate the force.

Let us now see in which category our system falls. 

In [16]:
df = pd.read_csv("data_cvs/Nbody_disc.csv")         # data of the system
N = df.shape[0]                                     # number of objects of our problem

# creating a class with the mass, pos, vel of each element of the system
masses = df[['mass']]
positions = df[['x', 'y', 'z']]
velocities = df[['vx', 'vy', 'vz']]
masses = masses.values.reshape(-1)                                      # reshape mass array to 1D array (required by pyfalcon)
sys = Particles(positions.values, velocities.values, masses)            # using the class particles to define our system and use some useful tools

# applying our formulas written above
t_cross = np.max(sys.radius()/sys.vel_mod())        # maximum crossing time
t_simulation = 210                                  # given
t_relax = 0.1 * N/np.log(N)

# taking the ratio to compare
ratio = t_simulation/t_relax
print(ratio)

0.2417692270710541


/tmp/ipykernel_45994/1889176282.py:12: RuntimeWarning: invalid value encountered in divide
  t_cross = np.max(sys.radius()/sys.vel_mod())        # maximum crossing time


Therefore, we have $\frac{t_{simulation}}{t_{relax}} = 0.2418 \ll 1 \implies t_{simulation} \ll t_{relax} \implies$ COLLISIONLESS SYSTEM. \
Consequently it is possivle to reduce the complexity of the simulation treating the system as a fluid moving in phase space and neglecting collision (and formation of binaries as well).

# Force Estimate in Collisionless Simulations Based on Tree Algorithms

# Short Description of the Fireworks Package and its Implementation.